# Урок 2. Stacking.

Теперь поговорим о стекинге - другом популярном способе ансамблирования алгоритмов. Он принципиально отличается от бустинга двумя положениями. Первое, в стекинге обычно используются разнородные базовые алгоритмы (например, kNN, метод опорных векторов и дерево решений), тогда как в бустинге обычно используют ансамбли однородных алгоритмов (обычно применяются деревья решений). Второе, стекинг объединяет базовые алгоритмы, используя мета-алгоритм, который самостоятельно обучается на предоставленных базовыми моделями данных. Бустинг в свою очередь объединяет их, следуя детерминированному алгоритму.

Идея стекинга лежит на поверхности. Известно, что, если обучить несколько разных алгоритмов, то в задаче регрессии их среднее, а в задаче классификации — голосование по большинству, часто превосходят по качеству все эти алгоритмы по отдельности. Возникает вопрос: почему, собственно, нужно использовать для ансамблирования такие операции как усреднение или голосование? Можно же доверить ансамблироование очередному алгоритму (т.н. мета-алгоритму) машинного обучения, чтобы он сам решил, как ему действовать с полученными ответами базовых алгоритмов.

Чтобы понять, как работает классический стекинг, сначала разберем его простейшую реализацию - **блендинг (blending)**. Обучающую выборку делят на две части. 

![](https://248006.selcdn.ru/public/DS.%20Block%202.%20M9/blending_1.png)

На первой обучают базовые алгоритмы. Затем получают их ответы на второй части и на тестовой выборке. 

![](https://248006.selcdn.ru/public/DS.%20Block%202.%20M9/blending_2.png)

Понятно, что  ответ каждого алгоритма можно рассматривать как новый признак (т.н. метапризнак). На метапризнаках второй части обучения настраивают метаалгоритм. Затем запускают его на метапризнаках теста и получают ответ.

![](https://248006.selcdn.ru/public/DS.%20Block%202.%20M9/blending_3.png)

В такой реализации самый большой недостаток - деление обучающей выборки. Получается, что ни базовые алгоритмы, ни метаалгоритм не используют всего объёма обучения, а значит, мы можем пропустить важные данные.

Один из способов борьбы за использование всей обучающей выборки - реализация классического **стекинга (stacking)**. Понятно, что обучить базовые алгоритмы на всей обучающей выборке и потом для той же выборки построить метапризнаки нельзя: будет очевидное переобучение. Поэтому выборку разбивают на части (т.н. фолды, как в кросс-валидации).

![](https://248006.selcdn.ru/public/DS.%20Block%202.%20M9/meta_1.png)

Затем последовательно для каждого обучают базовые алгоритмы на всех фолдах, кроме одного, а на оставшемся получают ответы базовых алгоритмов и трактуют их как значения соответствующих метапризнаков на этом фолде.

![](https://248006.selcdn.ru/public/DS.%20Block%202.%20M9/meta_2.png)

Для получения метапризнаков объектов тестовой выборки базовые алгоритмы обучают на всей тренировочной выборке и берут их ответы на тестовой.

![](https://248006.selcdn.ru/public/DS.%20Block%202.%20M9/meta_3.png)

Для примера, если взять M базовых алгоритмов и обучить их на N объектах, то окончательная матрица метапризнаков для обучения мета-алгоритма будет выглядеть так:
![](https://248006.selcdn.ru/public/DS.%20Block%202.%20M9/meta_mtrx.png)

Отметим, что стекинг не всегда существенно повышает качество лучшего из базовых алгоритмов, однако если этот алгоритм убрать из базовых, то и качество стекинга не сильно падает.

Также возможно использовать стекинг с несколькими слоями, т.н. многоуровневый стекинг, т.е. ввести понятие мета-мета признака и мета-мета алгоритма и далее при желании расширить еще на несколько слоев. Такой метод часто применяется в соревнованиях по машинному обучению, где борьба идет за десятитысячные доли к точности модели, однако саму модель это сделает неинтерпретируемой, что плохо для бизнес-задач.

Вообще, в sklearn нет реализации стекинга, но существуют такие небазовые пакеты python как mlxtend или vecstack, в которых присутствует этот метод, которым можно воспользоваться "из коробки".

Мы же на примере базовой реализации блендинга покажем, как работает этот метод. В качестве базовых алгоритмов возьмем kNN, LogisticRegression, DesisionTreeClassifier и SupportVectorClassification, а в качестве мета-алгоритма будем использовать XGBoost.

Будем проверять работу стекинга на том же преобразованном датасете Titanic.

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np

In [0]:
titanic = pd.read_csv('titanic.csv')
targets = titanic.Survived
data = titanic.drop(columns='Survived')

x_train, x_test, y_train, y_test = train_test_split(data, 
                                                    targets,
                                                    train_size=0.8,
                                                    random_state=0)

Так как нам нужно и обучить базовые алгоритмы на тренировочном сете, и сделать на этих же данных предсказания для обучения мета-алгоритма, разделим тренировочные данные еще на два датасета: train и valid.

In [0]:
train, valid, train_true, valid_true = train_test_split(x_train, 
                                                        y_train,
                                                        train_size=0.5,
                                                        random_state=0)

Обучим базовые алгоритмы.

In [0]:
knn = KNeighborsClassifier(n_neighbors=3)
knn_model = knn.fit(train, train_true)

lr = LogisticRegression(random_state=17)
lr_model = lr.fit(train, train_true)

dtc = DecisionTreeClassifier(max_leaf_nodes=4, random_state=17)
dtc_model = lr.fit(train, train_true)

svc = SVC(random_state=17)
svc_model = svc.fit(train, train_true)

/home/impuling/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/impuling/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/impuling/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Теперь получим предсказания моделей для второй части тренировочных данных - valid, заполним получившимися метапризнаками матрицу для обучения мета-алгоритма и обучим его.

Заодно получим для каждого базового алгоритма метрику AUC для тестовых данных, которая покажет качество работы каждого алгоритма в отдельности.

In [0]:
models = [knn_model, lr_model, dtc_model, svc_model]
meta_mtrx = np.empty((valid.shape[0], len(models))) # (кол-во объектов, 4 алгоритма)

for n, model in enumerate(models):
    meta_mtrx[:, n] = model.predict(valid)
    predicted = model.predict(x_test)
    print(f'{n} auc: {roc_auc_score(y_test, predicted)}')
    
meta = XGBClassifier(n_estimators=40)
meta_model = meta.fit(meta_mtrx, valid_true)

0 auc: 0.7080368906455863
1 auc: 0.7777997364953887
2 auc: 0.7777997364953887
3 auc: 0.620223978919631


Получим метапризнаки базовых алгоритмов для тестовых данных, чтобы мета-алгоритм мог по ним сделать предсказания.

In [0]:
meta_mtrx_test = np.empty((x_test.shape[0], len(models))) 

for n, model in enumerate(models):
    meta_mtrx_test[:, n] = model.predict(x_test)
    
meta_predict = meta.predict(meta_mtrx_test)
print(f'Stacking AUC: {roc_auc_score(y_test, meta_predict)}')

Stacking AUC: 0.7777997364953887


Как мы видим, в этом случае финальный скор AUC не превышает результат лучших базовых алгоритмов. Попробуем убрать базовые алгоритмы с лучшими результатами и посмотреть, что изменится.

In [0]:
models = [knn_model, svc_model]
meta_mtrx = np.empty((valid.shape[0], len(models))) # (кол-во объектов, 4 алгоритма)

for n, model in enumerate(models):
    meta_mtrx[:, n] = model.predict(valid)
    predicted = model.predict(x_test)
    print(f'{n} auc: {roc_auc_score(y_test, predicted)}')

meta_model = meta.fit(meta_mtrx, valid_true)

meta_mtrx_test = np.empty((x_test.shape[0], len(models))) 

for n, model in enumerate(models):
    meta_mtrx_test[:, n] = model.predict(x_test)
    
meta_predict = meta.predict(meta_mtrx_test)
print(f'Stacking AUC: {roc_auc_score(y_test, meta_predict)}')

0 auc: 0.7080368906455863
1 auc: 0.620223978919631
Stacking AUC: 0.7233860342555994


Здесь же видно, что стекинг двух не самых лучших базовых алгоритмов дает результат выше, чем у каждого из них по отдельности.